# WorkWithMesh

this notebook contains several instructions into how to manipulate meshes in FeniCs, particularly on how to work with XML meshes, which despite being a legacy format, present certain convenience

In [ ]:
#Setup FeniCs on Colab

!wget "https://fem-on-colab.github.io/releases/fenics-install.sh" -O "/tmp/fenics-install.sh" && bash "/tmp/fenics-install.sh"
import dolfin
from google.colab import files

import platform, sys
python_version=platform.python_version()
from distutils.version import LooseVersion, StrictVersion

if ( LooseVersion(python_version) < LooseVersion("3.0.0")):
    print("Python3 is needed!");
    print("How to fix: Runtime/Change_runtime_type/Python 3");
    sys.exit()
try:
    from dolfin import *; from mshr import *
except ImportError as e:
    !apt-get install -y -qq software-properties-common python-software-properties module-init-tools
    !add-apt-repository -y ppa:fenics-packages/fenics
    !apt-get update -qq
    !apt install -y --no-install-recommends fenics
    from dolfin import *; from mshr import *

import matplotlib.pyplot as plt;
from IPython.display import clear_output, display; import time; import dolfin.common.plotting as fenicsplot
import time

import os, sys, shutil

dolfin_version = dolfin.__version__
print ('dolfin version:', dolfin_version)

!rm -rf * # clean up all files
# Useful commands
# Remove an empty folder      : os.rmdir("my_results")
# Remove a folder with files  : shutil.rmtree("results")
# Make a folder               : os.mkdir("my_results")
# Runtime/Change_runtime_type/Python3

In [ ]:
pip install meshio #Install meshio, mesh manipulation library

In [ ]:
#Import a .msh file from gmsh and transform it into a .xml mesh, which is compatible with fenics.

#warning: .msh meshes are generally done in 3d (taking the z component to be zero). This axis must be removed to work with 2d problems on FeniCs

import matplotlib.pyplot as plt
import meshio
from dolfin import *

msh = meshio.read("onlyNeedle.msh") #Name of the .msh file
meshio.write("onlyNeedle.xml",msh2)
mesh = Mesh("onlyNeedle.xml") #Creation of .xml file


cell_types = msh.cells

# Printing cell types
for entry in cell_types:
    cell_type = entry.type
    cell_data = entry.data
    print(f"Cell Type: {cell_type}, Number of Cells: {len(cell_data)}")



In [4]:
mesh = UnitSquareMesh(10,10) #Creation of a regular, structured mesh in FeniCs
mesh

In [ ]:
gdim = mesh.topology().dim() #Useful commands to check mesh dimensions
tdim = mesh.geometry().dim()

In [11]:
#How to save a mesh into a File
mesh = RectangleMesh(Point(0, 0), Point(1,1), 20, 20)
mesh_file = File("mesh.xml")
mesh_file << mesh

In [ ]:
# Uniform refinement: Refines a mesh equally in all of its domain
mesh = refine(mesh)
mesh

In [12]:
# Local refinement Modified. Refine a Mesh around a point

cell_markers = MeshFunction('bool',mesh,mesh.topology().dim())
cell_markers.set_all(False)
origin = Point(0.5 , 0.5) #Select point around which refinement should occur
for cell in cells(mesh):
  p = cell.midpoint()
  if p.distance(origin) < 0.25: #Select radius of circle of refinement
    cell_markers[cell] = True #Nodes inside the circle will be marked

mesh = refine(mesh, cell_markers) #Refine only marked nodes

mesh



In [ ]:
#Creation of a circular, structured mesh
meshlevel = 25
degree = 1
dim = 2
mesh = UnitDiscMesh.create(MPI.comm_world, meshlevel, degree, dim)

mesh